In [ ]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
import os

The Six Figure Salary Predictive Model based on School Choice (Classification)
Target Feature: Six Figure Salary by mid-career (True or False)
Descriptive features: School Name, Stem Percent, Total_Cost of School, Rank of School, Diversity Index 

The steps to preprocess the data for this model are, 
1. Filtering the data for null values in any cells and removing the entire row if so in the dataset, 
    diversity_school.csv, salary_potential.csv, and total_cost
2. Replace str cells with int for values to be used in calculations
3. Filter Regions using the states to create concentrated visualizations, remove any Unknown state categories
4. Once datasets are filtered place each necessary column to be used as features in a separate file modelling_data.csv

Step 1: Filter null values

Step 2: Replace the str cells with int values

Step 3: Filter Regions using the states to create concentrated visualizations, remove any unknown states

In [ ]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore


# Add Regions to Diversity_School dataset
def addRegions():
    # 4 Major Regions
    # States in the Northeast
    northeast_states = {'Connecticut', 'Delaware', 'Maine', 'Maryland', 'Massachusetts', 'New Hampshire', 'New Jersey', 'New York', 'Pennsylvania', 'Rhode Island', 'Vermont'}

    # States in the Midwest
    midwest_states = {'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Michigan', 'Minnesota', 'Missouri', 'Nebraska', 'North Dakota', 'Ohio', 'South Dakota', 'Wisconsin'}

    # States in the South
    south_states = {'Alabama', 'Arkansas', 'Florida', 'Georgia', 'Kentucky', 'Louisiana', 'Mississippi', 'North Carolina', 'Oklahoma', 'South Carolina', 'Tennessee', 'Texas', 'Virginia', 'West Virginia'}

    # States in the West
    west_states = {'Alaska', 'Arizona', 'California', 'Colorado', 'Hawaii', 'Idaho', 'Montana', 'Nevada', 'New Mexico', 'Oregon', 'Utah', 'Washington', 'Wyoming'}
    
    df = pd.read_csv('DS-Model/data_raw/diversity_school.csv', sep=',')

    
    for index, row in df.iterrows():
        current_region = ''
        if df.at[index,'state'] in northeast_states:
            current_region = 'northeast'
        elif df.at[index,'state'] in midwest_states:
            current_region = 'midwest'
        elif df.at[index,'state'] in south_states:
            current_region = 'south'
        elif df.at[index,'state'] in west_states:
            current_region = 'west'
        else:
            current_region = 'unknown'  
            
        df.at[index, 'region'] = current_region

        df.to_csv('DS-Model/data_raw/diversity_school.csv', index=False)

#addRegions()
# Now delete all rows with unknown regions
def cleanDiversitySchool():
    df = pd.read_csv('DS-Model/data_raw/diversity_school.csv', sep=',')
    print('Count of cells BEFORE dropping empty cells:', df.size, '\n')
    for index, row in df.iterrows():
        if df.at[index,'region'] == 'unknown':
            df = df.drop(index)
    df.dropna()
    df.to_csv('DS-Model/data_raw/diversity_school.csv', index=False)
    print('Count of cells AFTER dropping empty cells:', df.size, '\n')
# cleanDiversitySchool()

def addDiversityPercent():
    df = pd.read_csv('DS-Model/data_raw/diversity_school.csv', sep=',')
    
    for index, row in df.iterrows():
        diversity_percent = df.at[index, 'enrollment'] / df.at[index, 'total_enrollment']
            
        df.at[index,'diversity_percent'] = diversity_percent
    
    df.to_csv('DS-Model/data_raw/diversity_school.csv', index=False)
    
addDiversityPercent()

Step 4: Creat modelling_data.csv with all features and target 